In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import faiss
from langchain_community.document_loaders import PyPDFLoader
import pandas as pd
from PIL import Image
import base64
from langchain_core.messages import HumanMessage
import mimetypes


In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash",temprature = 0)


In [ ]:
class UniversalLoader:
    def __init__(self,llm):
        self.llm = llm

    def process_file(self, file_path: str):
        """
        Traffic Controller: Routes files to the correct reader.
        """
        # 1. Get extension and mime type
        _, ext = os.path.splitext(file_path)
        ext = ext.lower()
        mime_type, _ = mimetypes.guess_type(file_path)
        
        # 2. DEFINE CODE EXTENSIONS (Treat these as text)
        code_extensions = {'.py', '.js', '.ts', '.html', '.css', '.java', '.cpp', '.c', '.h', '.sql', '.md', '.json', '.xml', '.yaml', '.yml', '.txt'}

        if ext in code_extensions:
            return self._process_code(file_path, ext)
        
        elif mime_type and "pdf" in mime_type:
            return self._process_pdf(file_path)
        
        elif mime_type and "csv" in mime_type:
            return self._process_csv(file_path)
        
        elif mime_type and "image" in mime_type:
            return self._process_image(file_path)
        
        else:
            return f"Unsupported file type: {mime_type or ext}"
        
    def _process_code(self, file_path, ext):
        """Reads code files and wraps them in markdown."""
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()
            # Wrap in markdown so LLM knows it's code
            lang_map = {'.py': 'python', '.js': 'javascript', '.ts': 'typescript', '.html': 'html', '.sql': 'sql', '.css': 'css'}
            language = lang_map.get(ext, '')
            return f"```{language}\n{content}\n```"
        except Exception as e:
            return f"Error reading code file: {e}"
        
    def _process_txt(self,file_path):
        with open(file_path,'r') as f:
            return f.read()
        
    def _process_pdf(self,file_path):
        loader = PyPDFLoader(file_path)
        pages = loader.load_and_split()
        result = ""
        for page in pages:
            result += page.page_content + "\n"
        return result
    
    def _process_csv(self,file_path):
        df = pd.read_csv(file_path)
        return df.to_markdown(index=False)
    
    def _process_image(self,file_path):
        try:
            with open(file_path, 'rb') as f:
                    
                image_data = base64.b64encode(f.read()).decode('utf-8')
                
                Prompt = HumanMessage(content = [
                    {"type":"text","text":"Describe the following image in detail."},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}},
                ])
                response = self.llm.invoke([Prompt])
                return response.content

        except Exception as e:
            return "error processing image: " + str(e)
        

universalloader = UniversalLoader(model)
    
    



In [4]:
search_tool = DuckDuckGoSearchRun()
search_tool.name = "search_tool"
search_tool.description = "web search tool to find more information about a topic"

result = search_tool.invoke("where is udaipur?")

In [5]:
print(result)

Udaipur district is particularly rich in mineral resources as a large variety of important minerals are found here. So where is Udaipur & how did I get there … ? ... 1 thought on “ So where is Udaipur & how did I get there … ? ” Where is Udaipur in India? A Geographic Overview ... Udaipur is located in the northwestern Indian state of Rajasthan, approximately 415 km south of ... Uniquely different from other cities in the state, Udaipur is where you drop the pace, kick up your feet and commune with nature. Where is Udaipur , Rajasthan, India on Map? ... Road map of Udaipur , Rajasthan, India shows where the location is placed.


In [ ]:
tools = [search_tool]

model = model.bind_tools(tools)

